In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=2221ba97a79d594ec43bbe806ef9f4e22f1230fed57f97ff035bd54ace67030f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer

In [ ]:
spark = SparkSession.builder.appName('training').getOrCreate()

In [ ]:
spark

In [ ]:
agri_df = spark.read.csv("/content/drive/MyDrive/Data/agriculture.csv", header= True, inferSchema= True)

In [ ]:
prop_df = spark.read.option('header', 'true').csv("/content/drive/MyDrive/Data/Balanced_Property_data.csv", inferSchema= True)

In [ ]:
prop_df.printSchema()

root
 |-- VisitorID: integer (nullable = true)
 |-- Proj_Group: string (nullable = true)
 |-- Proj_Segment: string (nullable = true)
 |-- Proj_Type: string (nullable = true)
 |-- Visitor_Group: string (nullable = true)
 |-- Visitor_Gender: string (nullable = true)
 |-- Question_Age: string (nullable = true)
 |-- Question_Budget: double (nullable = true)
 |-- Question_HHIncome: double (nullable = true)
 |-- Question_InstallmentMonthly: double (nullable = true)
 |-- Question_MarriageStatus: string (nullable = true)
 |-- Question_Nationality: string (nullable = true)
 |-- Question_Occupation: string (nullable = true)
 |-- Question_P_Income: double (nullable = true)
 |-- Question_TimeToMoveIn: string (nullable = true)
 |-- Target_Variable_Convert: boolean (nullable = true)



In [ ]:
agri_df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- ItemCode: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- 1965: double (nullable = true)
 |-- 1966: double (nullable = true)
 |-- 1967: double (nullable = true)
 |-- 1968: double (nullable = true)
 |-- 1969: double (nullable = true)
 |-- 1970: double (nullable = true)
 |-- 1971: double (nullable = true)
 |-- 1972: double (nullable = true)
 |-- 1973: double (nullable = true)
 |-- 1974: double (nullable = true)
 |-- 1975: double (nullable = true)
 |-- 1976: double (nullable = true)
 |-- 1977: double (nullable = true)
 |-- 1978: double (nullable = true)
 |-- 1979: double (nullable = true)
 |-- 1980: double (nullable = true)
 |-- 1981: double (nullable = true)
 |-- 1982: double (nullable = true)
 |-- 1983: double (nullable = true)
 |-- 1984: double (nullable = true)
 |-- 1985: double (nullable = true)
 |-- 1986: double (nullable = true)
 |-- 1987: double (nullable = true)
 |-- 1988: 

In [ ]:
prop_df.columns

['VisitorID',
 'Proj_Group',
 'Proj_Segment',
 'Proj_Type',
 'Visitor_Group',
 'Visitor_Gender',
 'Question_Age',
 'Question_Budget',
 'Question_HHIncome',
 'Question_InstallmentMonthly',
 'Question_MarriageStatus',
 'Question_Nationality',
 'Question_Occupation',
 'Question_P_Income',
 'Question_TimeToMoveIn',
 'Target_Variable_Convert']

In [ ]:
prop_df.dtypes

[('VisitorID', 'int'),
 ('Proj_Group', 'string'),
 ('Proj_Segment', 'string'),
 ('Proj_Type', 'string'),
 ('Visitor_Group', 'string'),
 ('Visitor_Gender', 'string'),
 ('Question_Age', 'string'),
 ('Question_Budget', 'double'),
 ('Question_HHIncome', 'double'),
 ('Question_InstallmentMonthly', 'double'),
 ('Question_MarriageStatus', 'string'),
 ('Question_Nationality', 'string'),
 ('Question_Occupation', 'string'),
 ('Question_P_Income', 'double'),
 ('Question_TimeToMoveIn', 'string'),
 ('Target_Variable_Convert', 'boolean')]

In [ ]:
prop_df.select(['Question_Budget',
                "Question_HHIncome",
                "Question_InstallmentMonthly",
                "Question_P_Income"]).describe().show()

+-------+------------------+------------------+---------------------------+------------------+
|summary|   Question_Budget| Question_HHIncome|Question_InstallmentMonthly| Question_P_Income|
+-------+------------------+------------------+---------------------------+------------------+
|  count|            120000|            120000|                     120000|            120000|
|   mean| 7.399064734564415| 290739.1420503373|          54185.80925723917| 141604.7010900417|
| stddev|3.6159924620618655|138854.06601809332|         20877.699268523076|46490.751621559706|
|    min|               2.0|           40011.0|                    15022.0|           70000.0|
|    max|              17.0|        640301.875|                   107673.0|          257500.0|
+-------+------------------+------------------+---------------------------+------------------+



In [ ]:
temp_df = prop_df.select(['Question_Budget',
                "Question_HHIncome",
                "Question_InstallmentMonthly",
                "Question_P_Income"])

In [ ]:
new_temp_df = temp_df.withColumn("double_Income", temp_df["Question_P_Income"]*2)
new_temp_df.show(10)

+-----------------+------------------+---------------------------+------------------+------------------+
|  Question_Budget| Question_HHIncome|Question_InstallmentMonthly| Question_P_Income|     double_Income|
+-----------------+------------------+---------------------------+------------------+------------------+
|             11.3|          610326.0|                   100133.0|          208750.0|          417500.0|
|             13.5|          421363.0|                   103857.0|          225000.0|          450000.0|
|             11.3|          413276.5|                    79472.0|          192500.0|          385000.0|
|7.377227028973517|284728.25108687003|         52415.683805236346|132396.80457292788|264793.60914585576|
|             17.0|          588512.0|                   107673.0|          135000.0|          270000.0|
|             14.5|          308717.0|                    78257.0|          198750.0|          397500.0|
|             16.0|          412127.0|                 

In [ ]:
new_temp_df = new_temp_df.drop('double_Income')
new_temp_df.show()

+-----------------+------------------+---------------------------+------------------+
|  Question_Budget| Question_HHIncome|Question_InstallmentMonthly| Question_P_Income|
+-----------------+------------------+---------------------------+------------------+
|             11.3|          610326.0|                   100133.0|          208750.0|
|             13.5|          421363.0|                   103857.0|          225000.0|
|             11.3|          413276.5|                    79472.0|          192500.0|
|7.377227028973517|284728.25108687003|         52415.683805236346|132396.80457292788|
|             17.0|          588512.0|                   107673.0|          135000.0|
|             14.5|          308717.0|                    78257.0|          198750.0|
|             16.0|          412127.0|                   100777.5|          241250.0|
|             16.0|          596811.0|                   100777.5|          241250.0|
|             14.5|        640301.875|                

In [ ]:
col_names= new_temp_df.columns
new_col_names = [name.split("_")[-1] for name in col_names]
new_col_names

['Budget', 'HHIncome', 'InstallmentMonthly', 'Income']

In [ ]:
()

In [ ]:
new_temp_df = new_temp_df.withColumnRenamed("Question_Budget", "Budget")
new_temp_df.show(10)

+-----------------+------------------+---------------------------+------------------+
|           Budget| Question_HHIncome|Question_InstallmentMonthly| Question_P_Income|
+-----------------+------------------+---------------------------+------------------+
|             11.3|          610326.0|                   100133.0|          208750.0|
|             13.5|          421363.0|                   103857.0|          225000.0|
|             11.3|          413276.5|                    79472.0|          192500.0|
|7.377227028973517|284728.25108687003|         52415.683805236346|132396.80457292788|
|             17.0|          588512.0|                   107673.0|          135000.0|
|             14.5|          308717.0|                    78257.0|          198750.0|
|             16.0|          412127.0|                   100777.5|          241250.0|
|             16.0|          596811.0|                   100777.5|          241250.0|
|             14.5|        640301.875|                

In [ ]:
new_temp_df = new_temp_df.toDF('Budget', 'HHIncome', 'InstallmentMonthly', 'Income')
new_temp_df.show(10)

+-----------------+------------------+------------------+------------------+
|           Budget|          HHIncome|InstallmentMonthly|            Income|
+-----------------+------------------+------------------+------------------+
|             11.3|          610326.0|          100133.0|          208750.0|
|             13.5|          421363.0|          103857.0|          225000.0|
|             11.3|          413276.5|           79472.0|          192500.0|
|7.377227028973517|284728.25108687003|52415.683805236346|132396.80457292788|
|             17.0|          588512.0|          107673.0|          135000.0|
|             14.5|          308717.0|           78257.0|          198750.0|
|             16.0|          412127.0|          100777.5|          241250.0|
|             16.0|          596811.0|          100777.5|          241250.0|
|             14.5|        640301.875|          107673.0|          208750.0|
|             17.0|          251110.0|           39644.0|          135000.0|

In [ ]:
prop_df.show(10)

+---------+----------+------------+--------------+-------------+--------------+------------+-----------------+------------------+---------------------------+-----------------------+--------------------+--------------------+------------------+---------------------+-----------------------+
|VisitorID|Proj_Group|Proj_Segment|     Proj_Type|Visitor_Group|Visitor_Gender|Question_Age|  Question_Budget| Question_HHIncome|Question_InstallmentMonthly|Question_MarriageStatus|Question_Nationality| Question_Occupation| Question_P_Income|Question_TimeToMoveIn|Target_Variable_Convert|
+---------+----------+------------+--------------+-------------+--------------+------------+-----------------+------------------+---------------------------+-----------------------+--------------------+--------------------+------------------+---------------------+-----------------------+
|        1|     Outer|           B|Standard Units|      Central|             F|        51 +|             11.3|          610326.0|    

In [ ]:
prop_df.filter("Proj_Group == 'Outer'").filter("Visitor_Group == 'Central'").filter("Visitor_Gender == 'F'").show(10)

+---------+----------+------------+--------------+-------------+--------------+------------+-----------------+------------------+---------------------------+-----------------------+--------------------+--------------------+------------------+---------------------+-----------------------+
|VisitorID|Proj_Group|Proj_Segment|     Proj_Type|Visitor_Group|Visitor_Gender|Question_Age|  Question_Budget| Question_HHIncome|Question_InstallmentMonthly|Question_MarriageStatus|Question_Nationality| Question_Occupation| Question_P_Income|Question_TimeToMoveIn|Target_Variable_Convert|
+---------+----------+------------+--------------+-------------+--------------+------------+-----------------+------------------+---------------------------+-----------------------+--------------------+--------------------+------------------+---------------------+-----------------------+
|        1|     Outer|           B|Standard Units|      Central|             F|        51 +|             11.3|          610326.0|    

In [ ]:
prop_df.filter(prop_df["Proj_Group"]== 'Outer').show()

+---------+----------+------------+--------------+-------------+--------------+------------+---------------+-----------------+---------------------------+-----------------------+--------------------+--------------------+-----------------+---------------------+-----------------------+
|VisitorID|Proj_Group|Proj_Segment|     Proj_Type|Visitor_Group|Visitor_Gender|Question_Age|Question_Budget|Question_HHIncome|Question_InstallmentMonthly|Question_MarriageStatus|Question_Nationality| Question_Occupation|Question_P_Income|Question_TimeToMoveIn|Target_Variable_Convert|
+---------+----------+------------+--------------+-------------+--------------+------------+---------------+-----------------+---------------------------+-----------------------+--------------------+--------------------+-----------------+---------------------+-----------------------+
|        1|     Outer|           B|Standard Units|      Central|             F|        51 +|           11.3|         610326.0|                   

In [ ]:
prop_df.select(["Proj_Group", "Proj_Segment", "Proj_Type", "Visitor_Group", "Visitor_Gender", "Question_Budget", "Question_HHIncome"])\
        .withColumnRenamed("Question_Budget", "Budget")\
        .withColumnRenamed("Question_HHIncome", "Income")\
        .groupBy(["Proj_Group", "Proj_Segment"]).sum().show()

+----------+------------+------------------+--------------------+
|Proj_Group|Proj_Segment|       sum(Budget)|         sum(Income)|
+----------+------------+------------------+--------------------+
|   Central|           E| 2527.008405434603| 8.629961089130305E7|
|     Urban|           D| 44429.44836397448|1.7835447503868701E9|
|     Urban|           C| 37785.15424991981| 1.537841995647223E9|
|     Major|           B|  51137.7295027206| 2.216624874840502E9|
|     Other|           B|26314.354256002567| 1.011165302087955E9|
|   Central|           B|  5963.38606507121| 3.083213632402079E8|
|     Other|           C| 3914.776081519054| 1.935992994320454E8|
|     Urban|           A|30565.376437970204| 9.673890656989069E8|
|     Urban|           B| 95976.63241223726| 3.841384074109164E9|
|     Outer|           C| 109281.8666188958| 4.637986710809107E9|
|     Outer|           D|31366.709136585403|1.4250915340597174E9|
|     Major|           D| 31755.59026924872| 1.773883885760118E9|
|     Oute

In [ ]:
prop_df.select(['Question_Budget', "Question_HHIncome"]).show()

+-----------------+------------------+
|  Question_Budget| Question_HHIncome|
+-----------------+------------------+
|             11.3|          610326.0|
|             13.5|          421363.0|
|             11.3|          413276.5|
|7.377227028973517|284728.25108687003|
|             17.0|          588512.0|
|             14.5|          308717.0|
|             16.0|          412127.0|
|             16.0|          596811.0|
|             14.5|        640301.875|
|             17.0|          251110.0|
|              8.0|          124085.0|
|             17.0|          334896.0|
|              7.5|          148151.5|
|              6.5|          249237.0|
|             13.5|          580214.0|
|              2.5|        640301.875|
|              3.5|          237418.0|
|              9.5|          577864.0|
|             10.5|          370659.0|
|              5.5|          175722.0|
+-----------------+------------------+
only showing top 20 rows



In [ ]:
prop_df.select('Proj_Type', "Visitor_Gender").show()

+--------------+--------------+
|     Proj_Type|Visitor_Gender|
+--------------+--------------+
|Standard Units|             F|
|Standard Units|             M|
| Premium Units|             F|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             F|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             M|
| Premium Units|             M|
| Premium Units|             M|
|Standard Units|             M|
| Premium Units|             M|
| Premium Units|             M|
|Standard Units|             M|
|Standard Units|             M|
|Standard Units|             M|
+--------------+--------------+
only showing top 20 rows



In [ ]:
[row.__getitem__("Proj_Type") for row in prop_df.select('Proj_Type').distinct().collect()]

['Premium Units', 'Standard Units']

In [ ]:
[row.__getitem__("Visitor_Gender") for row in prop_df.select('Visitor_Gender').distinct().collect()]

['F', 'M', 'Not Answer']

In [ ]:
[row.__getitem__("Proj_Group") for row in prop_df.select('Proj_Group').distinct().collect()]

['Urban', 'Other', 'Central', 'Outer', 'Major']

In [ ]:
ord_encoder = StringIndexer(inputCol="Proj_Group", outputCol="Proj_Group_encoded")
model = ord_encoder.fit(prop_df.select("Proj_Group"))
output_1 = model.transform(prop_df.select("Proj_Group"))
onehot_encoder = OneHotEncoder(inputCol="Proj_Group_encoded", outputCol="Proj_Group_onehot", dropLast=False).fit(output_1)
output_2 = onehot_encoder.transform(output_1)
output_2.show()

+----------+------------------+-----------------+
|Proj_Group|Proj_Group_encoded|Proj_Group_onehot|
+----------+------------------+-----------------+
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|   Central|               4.0|    (5,[4],[1.0])|
|     Urban|               1.0|    (5,[1],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Urban|               1.0|    (5,[1],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Other|               3.0|    (5,[3],[1.0])|
|     Major|               2.0|    (5,[2],[1.0])|
|     Outer|               0.0|    (5,[0],[1.0])|
|     Major|               2.0|    (5,[2],[1.0])|
|     Major|               2.0|    (5,[2],[1.0])|
